In [1]:
# CSL Paper: Dimensional speech emotion recognition from acoustic and text
# Changelog:
# 2019-09-01: initial version
# 2019-10-06: optimizer MTL parameters with linear search (in progress)
# 2019-12-21: update file for CSL journal
#             this python use following feature: mean, std, silence

import numpy as np
import pickle
import pandas as pd

import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Masking, CuDNNLSTM, TimeDistributed, \
                         Bidirectional, Flatten, \
                         Embedding, Dropout, Flatten, BatchNormalization, \
                         RNN, concatenate, Activation, LSTM
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence

#from keras_self_attention import SeqSelfAttention
from keras.callbacks import EarlyStopping, ModelCheckpoint
# from attention_helper import AttentionDecoder
#from read_csv import load_features

import random as rn
import tensorflow as tf

# import gensim
# from gensim.models import Word2Vec
# from gensim.models.keyedvectors import KeyedVectors
import codecs

Using TensorFlow backend.


In [2]:
rn.seed(123)
np.random.seed(99)
tf.set_random_seed(1234)

In [3]:
# x_text = np.load('../improv_loso/data/x_text.npy')

In [4]:
# x_text.shape

In [5]:
# load data - label
list_path_train = '../improv_loso/data/data_mspin_train.csv'
list_train = pd.read_csv(list_path_train, index_col=None)

# vad_train = [list_train['v'], list_train['a'], list_train['d']]
# vad_train = np.array(vad_train).T

list_path_test = '../improv_loso/data/data_mspin_test.csv'
list_test = pd.read_csv(list_path_test, index_col=None)

## Text processing train
list_file = pd.concat([list_train, list_test])
text = list_file['transcript']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
word_index = tokenizer.word_index
nb_words = len(word_index) + 1
print('Found %s unique tokens' % len(word_index))

token_tr_X = tokenizer.texts_to_sequences(text)
MAX_SEQUENCE_LENGTH = len(max(token_tr_X, key=len))
x_text = []
x_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

# x_text = np.load('improv_loso/data/x_text.npy')
# x_train_text = x_text[:5744] 
# x_test_text = x_text[5744:]

# vad = np.load('improv_loso/data/vad.npy')
# vad_train = vad[:5744]
# vad_test = vad[5744:]

EMBEDDING_DIM = 300
# nb_words = 3438
# MAX_SEQUENCE_LENGTH = 300

#path = '/home/s1820002/IEMOCAP-Emotion-Detection/'
file_loc =  '../../../dimensionalSM/data/glove.42B.300d.txt'
print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 3427 unique tokens
../../../dimensionalSM/data/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 167


In [6]:
vad = np.load('../improv_loso/data/vad.npy')

In [7]:
# standardization
scaled_feature = False

# set Dropout
do = 0.3

# standardized label
scaled_vad = True

# standardization
if scaled_vad:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler = scaler.fit(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    scaled_vad = scaler.transform(vad) #.reshape(vad.shape[0]*vad.shape[1], vad.shape[2]))
    vad = scaled_vad 
else:
    vad = vad


# Concordance correlation coefficient (CCC)-based loss function
# using non-inductive statistics
def ccc(gold, pred):
    gold       = K.squeeze(gold, axis=-1)
    pred       = K.squeeze(pred, axis=-1)
    gold_mean  = K.mean(gold, axis=-1, keepdims=True)
    pred_mean  = K.mean(pred, axis=-1, keepdims=True)
    covariance = (gold-gold_mean)*(pred-pred_mean)
    gold_var   = K.mean(K.square(gold-gold_mean), axis=-1,  keepdims=True)
    pred_var   = K.mean(K.square(pred-pred_mean), axis=-1, keepdims=True)
    ccc        = K.constant(2.) * covariance / (gold_var + pred_var + K.square(gold_mean - pred_mean) + K.common.epsilon())
    return ccc


def ccc_loss(gold, pred):  
    # input (num_batches, seq_len, 1)
    ccc_loss   = K.constant(1.) - ccc(gold, pred)
    return ccc_loss

In [8]:
# split train/test
split = 5744
earlystop = EarlyStopping(monitor='val_loss', patience=10, mode='min', 
                          restore_best_weights=True)

In [9]:
# API model, if use RNN, first two rnn layer must return_sequences=True
# model: GRU
def text_model1():
    inputs = Input(shape=(MAX_SEQUENCE_LENGTH, ))
    net = Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    trainable = True)(inputs)
    net = Bidirectional(LSTM(128, return_sequences=True))(net)
    net = Bidirectional(LSTM(64, return_sequences=False))(net)  
    out_text = Dense(64)(net)
    target_names = ('v', 'a', 'd')
    outputs = [Dense(1, name=name)(out_text) for name in target_names]

    model = Model(inputs=inputs, outputs=outputs) #=[out1, out2, out3])
    model.compile(loss=ccc_loss, #{'v': ccc_loss, 'a': ccc_loss, 'd': ccc_loss},
                     loss_weights={'v': 1, 'a': 1, 'd': 1},
                  optimizer='rmsprop', metrics=[ccc])
    return model


# model1 = text_model1()

In [10]:
# main function for for LOSO
def main(time):
    model_1 = text_model1()
    earlystop = EarlyStopping(monitor='val_loss', mode='min', patience=10,
                              restore_best_weights=True)
    hist = model_1.fit( [x_text[:split]],
                      vad[:split].T.tolist(), batch_size=32, #best:8
                      validation_split=0.2, epochs=50, verbose=1, shuffle=True,
                      callbacks=[earlystop])
    metrik = model_1.evaluate( [x_text[split:]], vad[split:].T.tolist())
    print(metrik[-3:])
    # make prediction
    predict = model_1.predict(x_text[4596:], batch_size=32)
    np.save('5744npy-glove/text_5744-glove-npy'+str(time)+'.npy',  
             np.array(predict).reshape(3, 2570).T)
    return metrik
    
if __name__ == '__main__':
    #loss_weight_step = np.around(np.arange(0.0, 1.1, 0.1), decimals=1)
    #dropout_rates = np.arange(0.5, 0.8, 0.1)
    weight = []
    #for i in dropout_rates:
    for time in range(20):
        best = main(time)
        weight.append([time, best[-3], best[-2], best[-1], np.mean(best[-3:])])
        print(time)
    weight_df = pd.DataFrame(weight)
    weight_df.to_csv('5744npy-glove/text-glove_5744.csv', header=['time', 'v','a','d', 'ave'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 94s 20ms/step - loss: 2.5656 - v_loss: 0.7244 - a_loss: 0.9020 - d_loss: 0.9355 - v_ccc: 0.2742 - a_ccc: 0.0965 - d_ccc: 0.0637 - val_loss: 2.6399 - val_v_loss: 0.8021 - val_a_loss: 0.9023 - val_d_loss: 0.9358 - val_v_ccc: 0.1976 - val_a_ccc: 0.0979 - val_d_ccc: 0.0646
Epoch 2/50
4595/4595 [==============================] - 91s 20ms/step - loss: 2.0596 - v_loss: 0.5762 - a_loss: 0.7175 - d_loss: 0.7666 - v_ccc: 0.4236 - a_ccc: 0.2830 - d_ccc: 0.2337 - val_loss: 2.6889 - val_v_loss: 0.8493 - val_a_loss: 0.9012 - val_d_loss: 0.9380 - val_v_ccc: 0.1503 - val_a_ccc: 0.0990 - val_d_ccc: 0.0617
Epoch 3/50
4595/4595 [==============================] - 91s 20ms/step - loss: 1.8239 - v_loss: 0.5022 - a_loss: 0.6356 - d_loss: 0.6861 - v_ccc: 0.4979 - a_ccc: 0.3644 - d_ccc: 0.3138 - val_loss: 2.5780 - val_v_loss: 0

Epoch 11/50
4595/4595 [==============================] - 89s 19ms/step - loss: 0.8093 - v_loss: 0.2358 - a_loss: 0.2809 - d_loss: 0.2923 - v_ccc: 0.7645 - a_ccc: 0.7188 - d_ccc: 0.7074 - val_loss: 2.7005 - val_v_loss: 0.8413 - val_a_loss: 0.9036 - val_d_loss: 0.9550 - val_v_ccc: 0.1578 - val_a_ccc: 0.0962 - val_d_ccc: 0.0455
Epoch 12/50
4595/4595 [==============================] - 90s 20ms/step - loss: 0.7456 - v_loss: 0.2140 - a_loss: 0.2585 - d_loss: 0.2725 - v_ccc: 0.7859 - a_ccc: 0.7414 - d_ccc: 0.7271 - val_loss: 2.6521 - val_v_loss: 0.8259 - val_a_loss: 0.8924 - val_d_loss: 0.9326 - val_v_ccc: 0.1731 - val_a_ccc: 0.1074 - val_d_ccc: 0.0674
Epoch 13/50
1422/1422 [==============================] - 8s 5ms/step
[0.22537733614444733, 0.21868346631526947, 0.11075711250305176]
1
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 95s 21ms/step - loss: 2.5711 - v_loss: 0.7115 - a_loss: 0.9127 - d_loss: 0.9470 - v_ccc: 0.2891 - a_ccc: 0.

4595/4595 [==============================] - 90s 20ms/step - loss: 0.7999 - v_loss: 0.2305 - a_loss: 0.2739 - d_loss: 0.2947 - v_ccc: 0.7694 - a_ccc: 0.7257 - d_ccc: 0.7050 - val_loss: 2.7223 - val_v_loss: 0.8655 - val_a_loss: 0.9278 - val_d_loss: 0.9273 - val_v_ccc: 0.1337 - val_a_ccc: 0.0720 - val_d_ccc: 0.0720
Epoch 12/50
1422/1422 [==============================] - 8s 6ms/step
[0.20845480263233185, 0.2157614678144455, 0.1568983644247055]
3
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 93s 20ms/step - loss: 2.5259 - v_loss: 0.7371 - a_loss: 0.8805 - d_loss: 0.9060 - v_ccc: 0.2622 - a_ccc: 0.1188 - d_ccc: 0.0931 - val_loss: 2.6718 - val_v_loss: 0.7803 - val_a_loss: 0.9245 - val_d_loss: 0.9679 - val_v_ccc: 0.2196 - val_a_ccc: 0.0759 - val_d_ccc: 0.0326
Epoch 2/50
4595/4595 [==============================] - 88s 19ms/step - loss: 2.0604 - v_loss: 0.5777 - a_loss: 0.7232 - d_loss: 0.7613 - v_ccc: 0.4230 - a_ccc: 0.2777 - d_ccc: 0

4595/4595 [==============================] - 92s 20ms/step - loss: 0.8994 - v_loss: 0.2593 - a_loss: 0.3126 - d_loss: 0.3283 - v_ccc: 0.7413 - a_ccc: 0.6875 - d_ccc: 0.6718 - val_loss: 2.7340 - val_v_loss: 0.8431 - val_a_loss: 0.9091 - val_d_loss: 0.9800 - val_v_ccc: 0.1561 - val_a_ccc: 0.0907 - val_d_ccc: 0.0192
Epoch 11/50
4595/4595 [==============================] - 91s 20ms/step - loss: 0.8198 - v_loss: 0.2322 - a_loss: 0.2883 - d_loss: 0.2994 - v_ccc: 0.7680 - a_ccc: 0.7118 - d_ccc: 0.7005 - val_loss: 2.7274 - val_v_loss: 0.8486 - val_a_loss: 0.9133 - val_d_loss: 0.9642 - val_v_ccc: 0.1507 - val_a_ccc: 0.0867 - val_d_ccc: 0.0352
Epoch 12/50
4595/4595 [==============================] - 91s 20ms/step - loss: 0.7431 - v_loss: 0.2173 - a_loss: 0.2575 - d_loss: 0.2677 - v_ccc: 0.7826 - a_ccc: 0.7422 - d_ccc: 0.7321 - val_loss: 2.7352 - val_v_loss: 0.8678 - val_a_loss: 0.9224 - val_d_loss: 0.9432 - val_v_ccc: 0.1312 - val_a_ccc: 0.0774 - val_d_ccc: 0.0562
Epoch 13/50
1422/1422 [========

4595/4595 [==============================] - 90s 20ms/step - loss: 0.8892 - v_loss: 0.2548 - a_loss: 0.3131 - d_loss: 0.3230 - v_ccc: 0.7461 - a_ccc: 0.6870 - d_ccc: 0.6777 - val_loss: 2.7374 - val_v_loss: 0.8434 - val_a_loss: 0.9274 - val_d_loss: 0.9652 - val_v_ccc: 0.1558 - val_a_ccc: 0.0723 - val_d_ccc: 0.0345
Epoch 11/50
4595/4595 [==============================] - 90s 20ms/step - loss: 0.8045 - v_loss: 0.2318 - a_loss: 0.2836 - d_loss: 0.2914 - v_ccc: 0.7687 - a_ccc: 0.7178 - d_ccc: 0.7090 - val_loss: 2.7722 - val_v_loss: 0.8371 - val_a_loss: 0.9311 - val_d_loss: 1.0036 - val_v_ccc: 0.1625 - val_a_ccc: 0.0690 - val_d_ccc: -0.0036
Epoch 12/50
1422/1422 [==============================] - 8s 5ms/step
[0.25772303342819214, 0.23375104367733002, 0.21026626229286194]
7
Train on 4595 samples, validate on 1149 samples
Epoch 1/50
4595/4595 [==============================] - 95s 21ms/step - loss: 2.5638 - v_loss: 0.7299 - a_loss: 0.8948 - d_loss: 0.9372 - v_ccc: 0.2696 - a_ccc: 0.1044 - d_cc

...

4595/4595 [==============================] - 90s 19ms/step - loss: 0.9720 - v_loss: 0.2779 - a_loss: 0.3407 - d_loss: 0.3550 - v_ccc: 0.7230 - a_ccc: 0.6601 - d_ccc: 0.6448 - val_loss: 2.7748 - val_v_loss: 0.8506 - val_a_loss: 0.9451 - val_d_loss: 0.9777 - val_v_ccc: 0.1486 - val_a_ccc: 0.0545 - val_d_ccc: 0.0221
Epoch 10/50
4595/4595 [==============================] - 89s 19ms/step - loss: 0.8899 - v_loss: 0.2523 - a_loss: 0.3094 - d_loss: 0.3275 - v_ccc: 0.7477 - a_ccc: 0.6902 - d_ccc: 0.6723 - val_loss: 2.7181 - val_v_loss: 0.8615 - val_a_loss: 0.9295 - val_d_loss: 0.9260 - val_v_ccc: 0.1377 - val_a_ccc: 0.0704 - val_d_ccc: 0.0738
Epoch 11/50
4595/4595 [==============================] - 89s 19ms/step - loss: 0.8116 - v_loss: 0.2271 - a_loss: 0.2845 - d_loss: 0.2995 - v_ccc: 0.7725 - a_ccc: 0.7154 - d_ccc: 0.7005 - val_loss: 2.8181 - val_v_loss: 0.8552 - val_a_loss: 0.9680 - val_d_loss: 0.9940 - val_v_ccc: 0.1441 - val_a_ccc: 0.0318 - val_d_ccc: 0.0060
Epoch 12/50
1422/1422 [========

4595/4595 [==============================] - 91s 20ms/step - loss: 0.9527 - v_loss: 0.2679 - a_loss: 0.3351 - d_loss: 0.3492 - v_ccc: 0.7319 - a_ccc: 0.6647 - d_ccc: 0.6508 - val_loss: 2.8041 - val_v_loss: 0.8921 - val_a_loss: 0.9100 - val_d_loss: 1.0005 - val_v_ccc: 0.1069 - val_a_ccc: 0.0897 - val_d_ccc: -7.2094e-04
Epoch 10/50
4595/4595 [==============================] - 90s 20ms/step - loss: 0.8905 - v_loss: 0.2501 - a_loss: 0.3127 - d_loss: 0.3269 - v_ccc: 0.7498 - a_ccc: 0.6870 - d_ccc: 0.6728 - val_loss: 2.7181 - val_v_loss: 0.8599 - val_a_loss: 0.9098 - val_d_loss: 0.9470 - val_v_ccc: 0.1390 - val_a_ccc: 0.0900 - val_d_ccc: 0.0529
Epoch 11/50
4595/4595 [==============================] - 90s 20ms/step - loss: 0.7998 - v_loss: 0.2286 - a_loss: 0.2755 - d_loss: 0.2968 - v_ccc: 0.7722 - a_ccc: 0.7248 - d_ccc: 0.7033 - val_loss: 2.7627 - val_v_loss: 0.8404 - val_a_loss: 0.9240 - val_d_loss: 0.9968 - val_v_ccc: 0.1585 - val_a_ccc: 0.0755 - val_d_ccc: 0.0033
Epoch 12/50
1422/1422 [===

4595/4595 [==============================] - 89s 19ms/step - loss: 1.1218 - v_loss: 0.3189 - a_loss: 0.3953 - d_loss: 0.4085 - v_ccc: 0.6815 - a_ccc: 0.6047 - d_ccc: 0.5920 - val_loss: 2.6804 - val_v_loss: 0.8185 - val_a_loss: 0.9053 - val_d_loss: 0.9551 - val_v_ccc: 0.1806 - val_a_ccc: 0.0944 - val_d_ccc: 0.0446
Epoch 9/50
4595/4595 [==============================] - 90s 19ms/step - loss: 1.0045 - v_loss: 0.2883 - a_loss: 0.3508 - d_loss: 0.3666 - v_ccc: 0.7116 - a_ccc: 0.6500 - d_ccc: 0.6339 - val_loss: 2.7225 - val_v_loss: 0.8510 - val_a_loss: 0.9066 - val_d_loss: 0.9632 - val_v_ccc: 0.1478 - val_a_ccc: 0.0931 - val_d_ccc: 0.0365
Epoch 10/50
4595/4595 [==============================] - 90s 20ms/step - loss: 0.9208 - v_loss: 0.2652 - a_loss: 0.3200 - d_loss: 0.3364 - v_ccc: 0.7348 - a_ccc: 0.6801 - d_ccc: 0.6643 - val_loss: 2.7622 - val_v_loss: 0.8576 - val_a_loss: 0.9263 - val_d_loss: 0.9769 - val_v_ccc: 0.1415 - val_a_ccc: 0.0733 - val_d_ccc: 0.0230
Epoch 11/50
4595/4595 [=========

4595/4595 [==============================] - 87s 19ms/step - loss: 1.1752 - v_loss: 0.3300 - a_loss: 0.4089 - d_loss: 0.4357 - v_ccc: 0.6698 - a_ccc: 0.5909 - d_ccc: 0.5641 - val_loss: 2.6761 - val_v_loss: 0.8098 - val_a_loss: 0.9232 - val_d_loss: 0.9422 - val_v_ccc: 0.1895 - val_a_ccc: 0.0769 - val_d_ccc: 0.0575
Epoch 8/50
4595/4595 [==============================] - 88s 19ms/step - loss: 1.0653 - v_loss: 0.3073 - a_loss: 0.3734 - d_loss: 0.3873 - v_ccc: 0.6938 - a_ccc: 0.6276 - d_ccc: 0.6134 - val_loss: 2.7173 - val_v_loss: 0.8580 - val_a_loss: 0.8997 - val_d_loss: 0.9585 - val_v_ccc: 0.1412 - val_a_ccc: 0.1003 - val_d_ccc: 0.0413
Epoch 9/50
4595/4595 [==============================] - 87s 19ms/step - loss: 0.9738 - v_loss: 0.2780 - a_loss: 0.3394 - d_loss: 0.3569 - v_ccc: 0.7219 - a_ccc: 0.6611 - d_ccc: 0.6432 - val_loss: 2.7271 - val_v_loss: 0.8048 - val_a_loss: 0.9430 - val_d_loss: 0.9786 - val_v_ccc: 0.1946 - val_a_ccc: 0.0572 - val_d_ccc: 0.0211
Epoch 10/50
4595/4595 [==========

4595/4595 [==============================] - 89s 19ms/step - loss: 1.4678 - v_loss: 0.4074 - a_loss: 0.5154 - d_loss: 0.5454 - v_ccc: 0.5923 - a_ccc: 0.4852 - d_ccc: 0.4547 - val_loss: 2.6725 - val_v_loss: 0.8101 - val_a_loss: 0.9162 - val_d_loss: 0.9452 - val_v_ccc: 0.1892 - val_a_ccc: 0.0837 - val_d_ccc: 0.0547
Epoch 6/50
4595/4595 [==============================] - 89s 19ms/step - loss: 1.3211 - v_loss: 0.3646 - a_loss: 0.4633 - d_loss: 0.4945 - v_ccc: 0.6362 - a_ccc: 0.5367 - d_ccc: 0.5060 - val_loss: 2.7049 - val_v_loss: 0.8607 - val_a_loss: 0.9129 - val_d_loss: 0.9306 - val_v_ccc: 0.1385 - val_a_ccc: 0.0873 - val_d_ccc: 0.0693
Epoch 7/50
4595/4595 [==============================] - 89s 19ms/step - loss: 1.1812 - v_loss: 0.3304 - a_loss: 0.4163 - d_loss: 0.4379 - v_ccc: 0.6702 - a_ccc: 0.5848 - d_ccc: 0.5637 - val_loss: 2.7269 - val_v_loss: 0.8349 - val_a_loss: 0.9213 - val_d_loss: 0.9696 - val_v_ccc: 0.1642 - val_a_ccc: 0.0785 - val_d_ccc: 0.0303
Epoch 8/50
4595/4595 [===========

In [11]:
# hist1 = model1.fit(x_text[:split], vad[:split].T.tolist(), epochs=50, 
#                    batch_size=32, verbose=1, validation_split=0.2, 
#                    callbacks=[earlystop])

# eval_metrik1 = model1.evaluate(x_text[split:], vad[split:].T.tolist())
# print(eval_metrik1[-3:])

# # make prediction
# predict = model1.predict(x_text[4596:], batch_size=32)
# np.save('text_glove_npy_loso', 
#          np.array(predict).reshape(3, 2570).T)